<a href="https://colab.research.google.com/github/nasiksami/Deep-Learning/blob/main/Story_Generator_with_RNN_on_F_R_I_E_N_D_S_series_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/content/script/Friends_Transcript.txt'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import tensorflow as tf


In [ ]:
friends = '/content/script/Friends_Transcript.txt'

In [ ]:
text = open(friends, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it

print ('Length of text: {} characters'.format(len(text)))

Length of text: 4899189 characters


In [ ]:
#script =friends.read() #read the script file
import re
pattern = re.compile(r'\s(?=\w+(?=:))') # store the regex
result = re.split(pattern, text) # split the script where our pattern matched (pink dot)

for item in result:
    split_line = item.split(':')
    try:
        character = split_line[0]
        speech = split_line[1]
    except:
        pass

In [ ]:
# Take a look at the first 250 characters in text
print(text[:250])

THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & David Crane
[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
Monica: There's nothing to tell! He's just some guy I work with!
Joey:


In [ ]:
# The unique characters in the file

vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

94 unique characters


In [ ]:
# process the text

char2index = {u:i for i, u in enumerate(vocab)}
index2char = np.array(vocab)
text_as_int = np.array([char2index[c] for c in text])

In [ ]:
print('{')
for char,_ in zip(char2index, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2index[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  '#' :   4,
  '$' :   5,
  '%' :   6,
  '&' :   7,
  "'" :   8,
  '(' :   9,
  ')' :  10,
  '*' :  11,
  '+' :  12,
  ',' :  13,
  '-' :  14,
  '.' :  15,
  '/' :  16,
  '0' :  17,
  '1' :  18,
  '2' :  19,
  ...
}


In [ ]:
# Show how the first 13 characters from the text are mapped to integers

print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'THE ONE WHERE' ---- characters mapped to int ---- > [53 41 38  1 48 47 38  1 56 41 38 51 38]


In [ ]:
# convert text vector to charcter indices

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
      print(index2char[i.numpy()])

T
H
E
 
O


In [ ]:
# convert character to sequence

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(index2char[item.numpy()])))

'THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\nWritten by: Marta Kauffman & Da'
"vid Crane\n[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]\nMonica: There's nothin"
"g to tell! He's just some guy I work with!\nJoey: C'mon, you're going out with the guy! There's gotta "
'be something wrong with him!\nChandler: All right Joey, be nice. So does he have a hump? A hump and a '
"hairpiece?\nPhoebe: Wait, does he eat chalk?\n(They all stare, bemused.)\nPhoebe: Just, 'cause, I don't "


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [ ]:
# print input and target values

for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(index2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(index2char[target_example.numpy()])))

Input data:  'THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\nWritten by: Marta Kauffman & D'
Target data: 'HE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)\nWritten by: Marta Kauffman & Da'


In [ ]:
# Batch size
BATCH_SIZE = 64
BUFFER_SIZE = 10000
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [ ]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 256

In [ ]:
# build the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[BATCH_SIZE, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           12032     
_________________________________________________________________
gru (GRU)                    (64, None, 256)           296448    
_________________________________________________________________
dense (Dense)                (64, None, 94)            24158     
Total params: 332,638
Trainable params: 332,638
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 94) # (batch_size, sequence_length, vocab_size)


In [ ]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 94)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.54306


In [ ]:
model.compile(optimizer='adam', loss=loss)

In [ ]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'

# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(dataset, epochs=10, callbacks=[checkpoint_callback])


Epoch 1/10
757/757 [==============================] - 15s 19ms/step - loss: 2.0516
Epoch 2/10
757/757 [==============================] - 15s 19ms/step - loss: 1.4549
Epoch 3/10
757/757 [==============================] - 14s 19ms/step - loss: 1.3384
Epoch 4/10
757/757 [==============================] - 15s 20ms/step - loss: 1.2879
Epoch 5/10
757/757 [==============================] - 15s 19ms/step - loss: 1.2585
Epoch 6/10
757/757 [==============================] - 15s 19ms/step - loss: 1.2382
Epoch 7/10
757/757 [==============================] - 15s 19ms/step - loss: 1.2239
Epoch 8/10
757/757 [==============================] - 14s 19ms/step - loss: 1.2126
Epoch 9/10
757/757 [==============================] - 14s 19ms/step - loss: 1.2033
Epoch 10/10
757/757 [==============================] - 14s 19ms/step - loss: 1.1959


In [ ]:
# to keep prediction step simple we use batch size of 1. for that we need to rebuild model
# and restore weight from checkpoint

tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [ ]:
# rebuild the model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[1, None]),
    tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
])
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (1, None, 128)            12032     
_________________________________________________________________
gru_2 (GRU)                  (1, None, 256)            296448    
_________________________________________________________________
dense_2 (Dense)              (1, None, 94)             24158     
Total params: 332,638
Trainable params: 332,638
Non-trainable params: 0
_________________________________________________________________


In [ ]:
def generate_text(model, start_string):
    # Number of characters to generate
    num_generate = 5000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2index[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(index2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
print(generate_text(model, start_string=u"Chandler: "))

Chandler: (entering). I didn't tell you, tell me anything to make since I could make a crush out, moms ideas?
Phoebe: I never totally never haft the morning! Ffoctomst, I mean you know what? I'm so youre a lovers! (Ross and Phoebe smite officals interesting.
Monica: Come on! You are so great!
Phoebe: Count The Woman, the face and, and there's really like that...E memony like how memballs ment that woman to meet you Dut sitting in versian!
Ross: Oh... you know what? 
Joey to hold the way the camera.)
Gavinita: I'm tyoing careful and hes slowly looking get to stay center early no furlotming a pause> Live it I know those kind, I don't know what I see me... holding about this? (you ware! (Kathy phone) Hmmm... (checks it upsure in it!)
Rachel: Ok ya-2-7-7.
[Scene: Central Perk. Monica is on a posion beach tocan re like to help your mom. (Listens) Comportant, how wow! (She go-two houses.)
Ross: No, no, no, no! Not! Thats atricf cut so he should tell ...Mast let me feel birt.
Joey: Yeah-ho-ho

LETS TRY WITH LSTM

In [ ]:
#Encoding
#convert it into integer. All the characters.

vocab = sorted(set(text))  #to sort all the unique characters
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}  #mapping
idx2char = np.array(vocab) #making list or array to use the indexes

def text_to_int(text):
  return np.array([char2idx[c] for c in text]) #convert every single characters from the text into their integer representation 

text_as_int = text_to_int(text)

In [ ]:
# lets look at how part of our text is encoded
print("Text:", text[:13])
print("Encoded:", text_to_int(text[:13]))

Text: THE ONE WHERE
Encoded: [53 41 38  1 48 47 38  1 56 41 38 51 38]


In [ ]:
def int_to_text(ints):
  try:
    ints = ints.numpy() #if its not already a numpy array, it will make it in numpy array
  except:
    pass
  return ''.join(idx2char[ints])

print(int_to_text(text_as_int[:133]))

THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & David Crane
[Scene: Central Perk, 


In [ ]:
#training
#whatever input we have, we will train in such a way that the output is one character ahead
#for example We will Input Cana the O/P will be anad
seq_length = 100  # length of sequence for a training example
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int) #converts string to characters

In [ ]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

In [ ]:
#split input target
def split_input_target(chunk):  # for the example: hello
    input_text = chunk[:-1]  # hell
    target_text = chunk[1:]  # ello
    return input_text, target_text  # hell, ello

dataset = sequences.map(split_input_target)  # we use map to apply the above function to every entry

In [ ]:
for x, y in dataset.take(2):
  print("\n\nEXAMPLE\n")
  print("INPUT")
  print(int_to_text(x))
  print("\nOUTPUT")
  print(int_to_text(y))



EXAMPLE

INPUT
THE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & D

OUTPUT
HE ONE WHERE MONICA GETS A NEW ROOMATE (THE PILOT-THE UNCUT VERSION)
Written by: Marta Kauffman & Da


EXAMPLE

INPUT
vid Crane
[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
Monica: There's nothi

OUTPUT
id Crane
[Scene: Central Perk, Chandler, Joey, Phoebe, and Monica are there.]
Monica: There's nothin


In [ ]:
#Batch here means 64 different sequences of training examples
BATCH_SIZE = 64
VOCAB_SIZE = len(vocab)  # vocab is number of unique characters
EMBEDDING_DIM = 256
RNN_UNITS = 1024

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

data = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

Building the Model

Now it is time to build the model. We will use an embedding layer a LSTM and one dense layer that contains a node for each unique character in our training data. The dense layer will give us a probability distribution over all nodes.



In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
    tf.keras.layers.LSTM(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  return model

model = build_model(VOCAB_SIZE,EMBEDDING_DIM, RNN_UNITS, BATCH_SIZE)
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (64, None, 256)           24064     
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          5246976   
_________________________________________________________________
dense_3 (Dense)              (64, None, 94)            96350     
Total params: 5,367,390
Trainable params: 5,367,390
Non-trainable params: 0
_________________________________________________________________


In [ ]:
for input_example_batch, target_example_batch in data.take(1):
  example_batch_predictions = model(input_example_batch)  # ask our model for a prediction on our first batch of training data (64 entries)
  print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")  # print out the output shape

(64, 100, 94) # (batch_size, sequence_length, vocab_size)


In [ ]:
# we can see that the predicition is an array of 64 arrays, one for each entry in the batch
print(len(example_batch_predictions))
print(example_batch_predictions)

64
tf.Tensor(
[[[-2.73887534e-03 -7.74494046e-03  2.20756093e-03 ...  2.57569831e-03
    1.33047835e-03 -2.62582838e-03]
  [-2.74408446e-03 -2.49624532e-03  4.26726742e-03 ...  7.97169283e-04
    5.12999203e-03  2.36386375e-04]
  [-3.34032625e-03 -1.25609303e-03  5.14965830e-03 ...  2.96361279e-03
    9.94541962e-03  1.88797712e-04]
  ...
  [ 3.49248154e-03  4.98982787e-04  5.17737819e-04 ... -4.29233164e-03
   -6.81733154e-03 -4.08193097e-03]
  [-1.44741393e-03  2.05711578e-03  2.24246364e-03 ... -1.59115309e-03
   -8.24465230e-03 -6.46302011e-03]
  [ 4.95924754e-03  5.02006523e-03 -4.56887903e-03 ... -1.93669507e-03
   -3.04073468e-03 -8.33479315e-03]]

 [[-4.86154482e-03 -1.41304731e-02  9.23144352e-03 ...  3.15065216e-03
   -9.40760225e-03  5.68275340e-03]
  [-4.17194236e-03 -1.05023384e-02  4.33431473e-03 ... -1.78608682e-03
   -1.12338988e-02  5.69357537e-03]
  [-3.24340817e-03 -1.28085008e-02  8.61454383e-03 ... -4.20396682e-03
   -5.60572091e-03  7.18792574e-03]
  ...
  [ 3.659

In [ ]:
# lets examine one prediction
pred = example_batch_predictions[0]
print(len(pred))
print(pred)
# notice this is a 2d array of length 100, where each interior array is the prediction for the next character at each time step

100
tf.Tensor(
[[-0.00273888 -0.00774494  0.00220756 ...  0.0025757   0.00133048
  -0.00262583]
 [-0.00274408 -0.00249625  0.00426727 ...  0.00079717  0.00512999
   0.00023639]
 [-0.00334033 -0.00125609  0.00514966 ...  0.00296361  0.00994542
   0.0001888 ]
 ...
 [ 0.00349248  0.00049898  0.00051774 ... -0.00429233 -0.00681733
  -0.00408193]
 [-0.00144741  0.00205712  0.00224246 ... -0.00159115 -0.00824465
  -0.00646302]
 [ 0.00495925  0.00502007 -0.00456888 ... -0.0019367  -0.00304073
  -0.00833479]], shape=(100, 94), dtype=float32)


In [ ]:
# and finally well look at a prediction at the first timestep
time_pred = pred[0]
print(len(time_pred))
print(time_pred)
# and of course its 65 values representing the probabillity of each character occuring next

94
tf.Tensor(
[-2.7388753e-03 -7.7449405e-03  2.2075609e-03  3.5514729e-03
  1.3080693e-02  8.4709842e-03 -1.2235443e-02  6.8866587e-03
  2.4864040e-03 -3.1110789e-03  1.5032735e-03 -4.9297898e-03
  6.9309631e-03  2.4491642e-04 -6.6305096e-03 -3.3801536e-03
 -5.6461506e-03 -1.6940153e-03  4.7120955e-03 -2.3632608e-03
 -1.8994850e-03  7.5719785e-03  7.5461199e-03  3.3869823e-03
 -5.9433701e-04 -6.2652864e-04  2.7200549e-03 -1.3934993e-03
 -2.1327515e-03  2.1732820e-03  1.8564388e-03 -1.9032671e-04
  2.9990377e-03  1.7745350e-03  7.1851769e-03 -8.4970528e-03
 -5.4296758e-03  2.8720561e-03 -1.1835566e-02  4.5500793e-03
 -1.1331175e-04 -9.4219651e-03  2.4219069e-03 -8.5970778e-03
 -4.5025405e-03 -7.2283372e-03  7.3333282e-04  6.0473252e-03
 -1.3558152e-03 -1.3976358e-05 -1.3622472e-02 -2.9441800e-03
  5.4334006e-03  2.1101958e-03  7.1236659e-03  6.1732512e-03
 -6.1560739e-03 -5.7636038e-03  1.1134675e-02 -4.8813135e-03
  2.0750761e-03  4.7808718e-03  1.3789454e-03 -3.6973692e-04
 -4.296528

In [ ]:
# If we want to determine the predicted character we need to sample the output distribution (pick a value based on probabillity)
sampled_indices = tf.random.categorical(pred, num_samples=1)

# now we can reshape that array and convert all the integers to numbers to see the actual characters
sampled_indices = np.reshape(sampled_indices, (1, -1))[0]
predicted_chars = int_to_text(sampled_indices)

predicted_chars  # and this is what the model predicted for training sequence 1

"*!*x:Y?F..6k=7zd*U{=[bEm5{5Wc;X+*mbNI>yud-X0:`6N/2Fi.L:!m:]}V9wN @E:X_4\nX3_0'yv>s`F}#gok]#?Q%zonil(C"

In [ ]:
def loss(labels, logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [ ]:
#completing the model
model.compile(optimizer='adam', loss=loss)

In [ ]:
#creating checkpoints
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [ ]:
history = model.fit(data, epochs=20, callbacks=[checkpoint_callback])

Epoch 1/20
757/757 [==============================] - 69s 91ms/step - loss: 1.3027
Epoch 2/20
757/757 [==============================] - 69s 91ms/step - loss: 1.2151
Epoch 3/20
757/757 [==============================] - 69s 91ms/step - loss: 1.1666
Epoch 4/20
757/757 [==============================] - 69s 91ms/step - loss: 1.1335
Epoch 5/20
757/757 [==============================] - 69s 91ms/step - loss: 1.1082
Epoch 6/20
757/757 [==============================] - 69s 91ms/step - loss: 1.0869
Epoch 7/20
757/757 [==============================] - 69s 91ms/step - loss: 1.0691
Epoch 8/20
757/757 [==============================] - 69s 91ms/step - loss: 1.0521
Epoch 9/20
757/757 [==============================] - 69s 91ms/step - loss: 1.0371
Epoch 10/20
757/757 [==============================] - 69s 91ms/step - loss: 1.0227
Epoch 11/20
757/757 [==============================] - 70s 92ms/step - loss: 1.0097
Epoch 12/20
757/757 [==============================] - 69s 92ms/step - loss: 0.9968
E

In [ ]:
#loading the model
model = build_model(VOCAB_SIZE, EMBEDDING_DIM, RNN_UNITS, batch_size=1)



In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [ ]:
#We can load any checkpoint we want by specifying the exact file to load

In [ ]:
checkpoint_num = 10
#model.load_weights(tf.train.load_checkpoint("./training_checkpoints/ckpt_" + str(checkpoint_num)))
model.build(tf.TensorShape([1, None]))

In [ ]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
  num_generate = 1500

  # Converting our start string to numbers (vectorizing)
  input_eval = [char2idx[s] for s in start_string]
  input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
  text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
  temperature = 1.0

  # Here batch size == 1
  model.reset_states()
  for i in range(num_generate):
      predictions = model(input_eval)
      # remove the batch dimension
    
      predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
      predictions = predictions / temperature
      predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
      input_eval = tf.expand_dims([predicted_id], 0)

      text_generated.append(idx2char[predicted_id])

  return (start_string + ''.join(text_generated))

In [ ]:
inp = input("Type a starting string: ")
print(generate_text(model, inp))

Type a starting string: chandler
chandler, get it! (She gives him the picture.)
CHANDLER: Wha...  EDDIE: Is some nice kitchen? 
RACHEL: You know, Ross. Ross, you just don't mean about how smart knowing it is about me, everythings hit by (Ross tackles the slive on the seat.) Okay, scohing to Monica) What locked me?! (He's leaving with her purse and smiles) Ah, oh, uh yeah, there's all for her, uhm... I have   jokes. Uh, I was a waitress.  So I can fit into private ruined.
Chandler: I can sprave that strings wed face for Phoebe. Phoebe the twins and stuck to guestily.) Well then, Im so sorry about this the hardware-sawce! So anyway we seen him in the hall, but it makes you feel now, but lets just  
Joey: Chandler? (Monica looks convincing near the set) Not every weird hair to see the cookie.) Ooh! Now I'm steaming, and if its an opportunity test I went through a wedding actor with no resum, kissing Charlie and a family to put speech.]
Chandler: Honey, that's the smell of secret opportuni